<a href="https://colab.research.google.com/github/Wpert/numerics/blob/main/numericsGauss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Постановка задачи всё та же, что и в прошлой лаборатной работе:

Дана система уравнений $Ax = b$, найти решение этой системы $x$.

Метод решения абсолютно школьный, его асимтотика $O(n^3)$.
Ничего абсолютно нового нет.

$\left(
  \begin{array}{cccc|c}
      a_{11} & a_{12} & \dots & a_{1n}  & b_1 \\
      a_{21} & a_{22} & \dots & a_{2n}  & b_2 \\
      \vdots &        & \ddots & \vdots & b_3 \\
      a_{11} & a_{12} & \dots & a_{1n}  & b_n \\
  \end{array}
\right) \Rightarrow
\left(
  \begin{array}{cccc|c}
      a'_{11} & a'_{12} & \dots  & a'_{1n}  & b'_1   \\
      0       & a'_{22} & \dots  & a'_{2n}  & b'_2   \\
      \vdots  &         & \ddots & \vdots   & \vdots \\
      0       & 0       & \dots  & a'_{1n}  & b'_n   \\
  \end{array}
\right)$

После чего можно легко понять ранг матрицы и, если возможно, найти едиственное решение системы уравнений.




In [33]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

def isEqual(x, y) -> bool:
  return np.isclose(x, y, rtol=1e-05, atol=1e-08, equal_nan=False)

def GenerateMatrixOnes(N: int) -> Tuple[np.ndarray, np.ndarray]:
  Matr = np.full((N, N), 0.5) - np.random.rand(N, N)
  A = np.dot(Matr.T, Matr)
  b = np.zeros((N, 1))
  for i in range(N):
    b[i] += np.sum(A[i, :])

  return (A, b)

def GenerateRandomMatrix(N: int) -> Tuple[np.ndarray, np.ndarray]:
  return ( np.random.randint(30, size=(N, N)), np.random.randint(30, size=(N, 1)) )

def GaussianElimination(A: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    C = np.array(A, dtype=float)
    d = np.array(b, dtype=float)

    N = len(b)
    for i in range(0, N - 1):
        if np.abs(C[i, i]) == 0:
            for k in range(i + 1, N):
                if np.abs(C[k, i]) > np.abs(C[i, i]):
                    C[[i, k]] = C[[k, i]]
                    d[[i, k]] = d[[k, i]]
                    break

        for j in range(i + 1, N):
            if (isEqual(C[i, i], 0)):
                m = 0
            else:
              m = C[j, i] / C[i, i]

            C[j, : ] = C[j, : ] - m * C[i, : ]
            d[j] = d[j] - m * d[i]

    return C, d

def StepBack(C: np.ndarray, d: np.ndarray) -> np.ndarray:
  N = d.size

  if (GetRank(C) < N):
    raise Exception("Система не имеет единственного решения.")

  x = np.zeros((N, 1))
  x[N - 1] = d[N - 1] / C[N - 1, N-1]

  for i in range(N - 2, -1, -1):
    sum_ = 0

    for j in range(i + 1, N):
      sum_ = sum_ + C[i, j] * x[j]

    x[i] = (d[i] - sum_) / C[i, i]

  return x

def GetRank(C: np.ndarray) -> int:
  N = C.shape[0]
  cntZeroRows = len(np.where(~C.any(axis=1))[0])
  return N - cntZeroRows;

Соотв применение метода Гаусса:

In [34]:
from sympy import Matrix, init_printing
init_printing()
# A = np.array([[1, 2, 3, 4],
#               [4, 8, 12, 16],
#               [3, 6, 9, 12],
#               [1, 1, 1, 1]])
# b = [[1],
#      [2],
#      [3],
#      [1]]

# Генерируем матрицу
N: int = 4
A, b = GenerateRandomMatrix(N)
# A, b = GenerateMatrixOnes(N)

# Выводим всю информацию на экран
print('Исходная матрица:')
display(Matrix(A))
print('Столбец свободных членов:')
display(Matrix(b))

# Применяем метод и находим решение x
C, d = GaussianElimination(A, b)

print("\n Результат работы программы: \n")

try:
  x = StepBack(C, d)

  print('Преобразованная матрица A:')
  display(Matrix(C))
  print('Найденное решение x:')
  display(Matrix(x))
  print(f'Ранг матрицы: {GetRank(C)}')

  det = np.prod(np.diagonal(C))
  rightDet = np.linalg.det(A)
  print(f"\n Правильный определитель матрицы = {rightDet}")
  print(f"\n Мой определитель матрицы = {det}")

  print("\n Проверка результата: [A]{x} - b =")
  display(Matrix(np.matmul(A, x) - b))

  # Тест на правильность
  assert(isEqual(rightDet, det))
except Exception as no_sol:
  print(no_sol.args)

Исходная матрица:


⎡16  3   3   7 ⎤
⎢              ⎥
⎢12  17  29  5 ⎥
⎢              ⎥
⎢8   13  9   21⎥
⎢              ⎥
⎣10  0   17  15⎦

Столбец свободных членов:


⎡3 ⎤
⎢  ⎥
⎢13⎥
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣5 ⎦


 Результат работы программы: 

Преобразованная матрица A:


⎡16.0   3.0          3.0               7.0       ⎤
⎢                                                ⎥
⎢ 0    14.75        26.75             -0.25      ⎥
⎢                                                ⎥
⎢ 0      0    -13.3559322033898  17.6949152542373⎥
⎢                                                ⎥
⎣ 0      0            0          35.1370558375635⎦

Найденное решение x:


⎡0.226271308870269 ⎤
⎢                  ⎥
⎢0.0127130887026871⎥
⎢                  ⎥
⎢0.392408263507657 ⎥
⎢                  ⎥
⎣-0.26224357122219 ⎦

Ранг матрицы: 4

 Правильный определитель матрицы = -110751.99999999991

 Мой определитель матрицы = -110752.0

 Проверка результата: [A]{x} - b =


⎡          0          ⎤
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢-1.77635683940025e-15⎥
⎢                     ⎥
⎣-8.88178419700125e-16⎦